In [2]:
import os
import json
from pathlib import Path
from chunkformer_vpb.training.tokenizer import normalize_vi

def generate_test_meta(base_dir: str):
    base = Path(base_dir)
    input_dir = base / "archive_2"
    transcript_dir = input_dir / "transcript_corrected"
    transcript_base = input_dir / "transcripts"  # ✅ Đường dẫn đúng cho base prediction
    wavs_dir = input_dir / "wavs"

    output_dir = base / "standard_test_2"
    # create output directory if it doesn't exist
    output_dir.mkdir(parents=True, exist_ok=True)
    output_path = output_dir / "test_meta.json"

    entries = []

    for txt_file in sorted(transcript_dir.glob("*.txt")):
        utt_id = txt_file.stem
        wav_file = wavs_dir / f"{utt_id}.wav"

        if not wav_file.exists():
            print(f"⚠️  Missing audio for {utt_id}")
            continue

        with open(txt_file, "r", encoding="utf-8") as f:
            text = f.read().strip()

        if not text:
            print(f"⚠️  Empty transcript for {utt_id}")
            continue

        # === Đường dẫn tương đối từ base_dir ===
        relative_audio_path = wav_file.relative_to(base)

        # === Đọc kết quả dự đoán từ mô hình base ===
        base_pred_file = transcript_base / f"{utt_id}.txt"
        if base_pred_file.exists():
            with open(base_pred_file, "r", encoding="utf-8") as pf:
                base_text = pf.read().strip()
        else:
            print(f"⚠️  Missing base prediction for {utt_id}")
            base_text = ""

        entry = {
            "utt_id": utt_id,
            "audio_path": str(relative_audio_path),
            "text": normalize_vi(text),
            "base_text": normalize_vi(base_text)
        }
        entries.append(entry)

    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(entries, f, ensure_ascii=False, indent=2)

    print(f"✅ Created {len(entries)} entries")
    print(f"📄 Saved to: {output_path}")

# 👉 Cách gọi:
if __name__ == "__main__":
    generate_test_meta("../../../vpb_dataset")


⚠️  Missing audio for .progress
⚠️  Empty transcript for E_huongds_D_2025-06-02_H_081820_088_CLID_0362802032___000000706___right___000001343
⚠️  Empty transcript for E_huongds_D_2025-06-02_H_091735_844_CLID_0813494717___000005954___right___000006206
⚠️  Empty transcript for E_huongds_D_2025-06-02_H_091735_844_CLID_0813494717___000041666___right___000042311
⚠️  Empty transcript for E_huongds_D_2025-06-02_H_092528_644_CLID_0904622980___000022402___right___000022622
⚠️  Empty transcript for E_huongds_D_2025-06-02_H_092528_644_CLID_0904622980___000076738___right___000077150
⚠️  Empty transcript for E_huongds_D_2025-06-02_H_092528_644_CLID_0904622980___000133666___right___000133854
⚠️  Empty transcript for E_huongds_D_2025-06-02_H_092528_644_CLID_0904622980___000151682___right___000151966
⚠️  Empty transcript for E_huongds_D_2025-06-02_H_092528_644_CLID_0904622980___000213602___right___000213982
⚠️  Empty transcript for E_huongds_D_2025-06-02_H_092528_644_CLID_0904622980___000307458___right

In [4]:
from chunkformer_vpb.training.train import evaluate_from_meta

evaluate_from_meta("../../../vpb_dataset/standard_test_2/test_meta.json")

📊 Tổng số mẫu: 2890
🎯 WER trung bình (sample avg): 34.48%
🌐 WER toàn cục   (global):     24.59%
